In [1]:
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukr_g2p"
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukroASR"
UKROASR_ROOT=/data/exp/kostya/ukroASR
# export PYTHONIOENCODING=utf-8

In [2]:
cd $UKROASR_ROOT

In [3]:
# cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh

In [4]:
train_cmd=run.pl

In [5]:
work_dir=/data/exp/kostya/ukroASR

In [6]:
data=$work_dir/data

In [7]:
exp=$work_dir/exp

In [8]:
cmd_prefix='cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh;'

# IMPORT DATASET

In [8]:
# import train.csv

In [13]:
dvc import -o data/raw_data/train.csv \
https://github.com/kosti4ka/ukrodataset.git train.csv

Importing 'train.csv (https://github.com/kosti4ka/ukrodataset.git)' -> 'data/raw_data/train.csv'
                                                                                
To track the changes with git, run:

	git add data/raw_data/train.csv.dvc


In [15]:
# import dev.csv

In [16]:
dvc import -o data/raw_data/dev.csv \
https://github.com/kosti4ka/ukrodataset.git dev.csv

Importing 'dev.csv (https://github.com/kosti4ka/ukrodataset.git)' -> 'data/raw_data/dev.csv'
                                                                                
To track the changes with git, run:

	git add data/raw_data/dev.csv.dvc


In [18]:
# import audio data itself

In [87]:
dvc import -o data/raw_data \
https://github.com/kosti4ka/ukrodataset.git audiobooks

Importing 'audiobooks (https://github.com/kosti4ka/ukrodataset.git)' -> 'data/raw_data/audiobooks'
                                                                                
To track the changes with git, run:

	git add data/raw_data/audiobooks.dvc


In [89]:
git add -f data/raw_data/audiobooks.dvc

# GENERATING DATA FOLDERS

In [34]:
# train data folder

In [8]:
dvc run \
-d $data/raw_data/train.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/train/text -o $data/train/utt2spk -o $data/train/segments -o $data/train/spk2utt -o $data/train/wav.scp \
-n train_data_dir \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/train.csv $data/train

Stage is cached, skipping                                                       


In [35]:
# dev data folder

In [38]:
dvc run \
-d $data/raw_data/dev.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/dev/text -o $data/dev/utt2spk -o $data/dev/segments -o $data/dev/spk2utt -o $data/dev/wav.scp \
-n dev_data_dir \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/dev.csv $data/dev

Running stage 'dev_data_dir' with command:                                      
	python3.6 /data/exp/kostya/ukroASR/gen_data_dir.py /data/exp/kostya/ukroASR/data/raw_data/dev.csv /data/exp/kostya/ukroASR/data/dev
/bin/sh: 18: ./cmd.sh: [[: not found
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/spk2utt is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/segments is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/wav.scp is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 879 utterances.
fix_data_dir.sh: old files are kept in /data/exp/kostya/ukroASR/data/dev/.backup
Adding stage 'dev_data_dir' in 'd

In [41]:
git add -f dvc.lock data/dev/.gitignore dvc.yaml

# GENERATING LOCAL/DICT

In [10]:
dvc run -f \
-d $data/train/text -d $UKROASR_ROOT/artifacts/text_plane -d $UKROASR_ROOT/artifacts/normal_fiction.txt -d $UKROASR_ROOT/artifacts/lexicon.txt \
-o $data/local/dict/vocab -o $data/local/dict/lexicon.txt \
-n local_dict \
$UKROASR_ROOT/prepare_dict.sh $data/local/dict $data/train/text $UKROASR_ROOT/artifacts/text_plane $UKROASR_ROOT/artifacts/normal_fiction.txt

Running stage 'local_dict' with command:                                        
	/data/exp/kostya/ukroASR/prepare_dict.sh /data/exp/kostya/ukroASR/data/local/dict /data/exp/kostya/ukroASR/data/train/text /data/exp/kostya/ukroASR/artifacts/text_plane /data/exp/kostya/ukroASR/artifacts/normal_fiction.txt
100%|##################################| 313994/313994 [49:03<00:00, 106.67it/s]
Modifying stage 'local_dict' in 'dvc.yaml'                                      
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [20]:
git add dvc.yaml dvc.lock

# Generating Kaldi's lang dir

In [49]:
dvc run \
-d $data/local/dict/lexicon.txt -d $data/local/dict/silence_phones.txt -d $data/local/dict/nonsilence_phones.txt -d $data/local/dict/optional_silence.txt -d $data/local/dict/extra_questions.txt \
-o $data/lang/L.fst -o $data/lang/L_disambig.fst -o $data/lang/oov.int -o $data/lang/oov.txt -o $data/lang/phones.txt -o $data/lang/topo -o $data/lang/words.txt -o $data/lang/phones -O $data/local/dict/lexiconp.txt \
-n lang_dir \
"$cmd_prefix utils/prepare_lang.sh --position-dependent-phones false $data/local/dict '<UNK>' $data/local/lang $data/lang"

Running stage 'lang_dir' with command:
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/prepare_lang.sh --position-dependent-phones false /data/exp/kostya/ukroASR/data/local/dict '<UNK>' /data/exp/kostya/ukroASR/data/local/lang /data/exp/kostya/ukroASR/data/lang
utils/prepare_lang.sh --position-dependent-phones false /data/exp/kostya/ukroASR/data/local/dict <UNK> /data/exp/kostya/ukroASR/data/local/lang /data/exp/kostya/ukroASR/data/lang
Checking /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt ...
--> reading /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt is OK

Checking /data/exp/kostya/ukroASR/data/local/dict/optional_silence.txt ...
--> reading /data/exp/kostya/ukroASR/data/local/dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only a

Checking /data/exp/kostya/ukroASR/data/lang/oov.{txt, int} ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> 1 entry/entries in /data/exp/kostya/ukroASR/data/lang/oov.txt
--> /data/exp/kostya/ukroASR/data/lang/oov.int corresponds to /data/exp/kostya/ukroASR/data/lang/oov.txt
--> /data/exp/kostya/ukroASR/data/lang/oov.{txt, int} are OK

--> /data/exp/kostya/ukroASR/data/lang/L.fst is olabel sorted
--> /data/exp/kostya/ukroASR/data/lang/L_disambig.fst is olabel sorted
--> SUCCESS [validating lang directory /data/exp/kostya/ukroASR/data/lang]
Modifying stage 'lang_dir' in 'dvc.yaml'                                        

To track the changes with git, run:

	git add dvc.yaml


In [50]:
git add dvc.yaml

# Generating LM

In [11]:
dvc run -f \
-d $data/train/text -d $UKROASR_ROOT/artifacts/text_plane -d $UKROASR_ROOT/artifacts/normal_fiction.txt \
-o $data/lm_ukr.gz \
-n lm_ukr \
"cut -d' ' -f2- $data/train/text $UKROASR_ROOT/artifacts/text_plane $UKROASR_ROOT/artifacts/normal_fiction.txt | ngram-count -text - -order 3 -gt1min 1 -gt2min 1 -lm - | gzip -c > $data/lm_ukr.gz"

Running stage 'lm_ukr' with command:                                            
	cut -d' ' -f2- /data/exp/kostya/ukroASR/data/train/text /data/exp/kostya/ukroASR/artifacts/text_plane /data/exp/kostya/ukroASR/artifacts/normal_fiction.txt | ngram-count -text - -order 3 -gt1min 1 -gt2min 1 -lm - | gzip -c > /data/exp/kostya/ukroASR/data/lm_ukr.gz
Modifying stage 'lm_ukr' in 'dvc.yaml'                                          
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [63]:
git add dvc.lock dvc.yaml

In [12]:
dvc run -f \
-d $data/dev/text -d $data/lm_ukr.gz \
-m $exp/metrics/lm.json \
-n lm_metrics \
"echo {\'LM perplexity\': $(cut -d' ' -f2- $data/dev/text | ngram -lm $data/lm_ukr.gz -ppl - | tail -1 | awk '{print $6}')} > $exp/metrics/lm.json"

Running stage 'lm_metrics' with command:                                        
	echo {\'LM perplexity\': 2486.735} > /data/exp/kostya/ukroASR/exp/metrics/lm.json
Modifying stage 'lm_metrics' in 'dvc.yaml'                                      
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [68]:
git add dvc.lock dvc.yaml

# Formatting Kaldi's LM dir

In [76]:
dvc run \
-d $data/lang/words.txt -d $data/lm_ukr.gz -d $data/local/dict/lexicon.txt \
-o $data/lang/G.fst \
-n lang_fst \
"$cmd_prefix utils/format_lm.sh $data/lang $data/lm_ukr.gz $data/local/dict/lexicon.txt $data/lang"

Running stage 'lang_fst' with command:                                          
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/format_lm.sh /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/data/lm_ukr.gz /data/exp/kostya/ukroASR/data/local/dict/lexicon.txt /data/exp/kostya/ukroASR/data/lang
Converting '/data/exp/kostya/ukroASR/data/lm_ukr.gz' to FST
arpa2fst --disambig-symbol=#0 --read-symbol-table=/data/exp/kostya/ukroASR/data/lang/words.txt - /data/exp/kostya/ukroASR/data/lang/G.fst 
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 8 [-5.162682	0	-0.1165124] skipped: word '0' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 9 [-6.681196	0-ой	-0.09000115] skipped: word '0-ой' not in symbol table
WARNING (arpa2fst[5.5.303~1-

In [13]:
l $data/lang

total 627M
-rw-r--r-- 1 ubuntu ubuntu    6 Jun 29 13:55 oov.txt
-rw-r--r-- 1 ubuntu ubuntu    2 Jun 29 13:55 oov.int
-rw-r--r-- 1 ubuntu ubuntu 1.3K Jun 29 13:55 topo
-rw-r--r-- 1 ubuntu ubuntu 183M Jun 29 13:59 L.fst
-rw-r--r-- 1 ubuntu ubuntu 187M Jun 29 13:59 L_disambig.fst
drwxrwxr-x 2 ubuntu ubuntu 4.0K Jun 29 13:59 phones
-rw-r--r-- 1 ubuntu ubuntu  824 Jun 29 13:59 phones.txt
-rw-r--r-- 1 ubuntu ubuntu  18M Jun 29 13:59 words.txt
-rw-r--r-- 1 ubuntu ubuntu 241M Jun 29 20:38 G.fst


In [14]:
rm -rf $data/lang/G.fst

In [77]:
git add dvc.yaml dvc.lock

# Computing features

In [91]:
dvc run -f \
-d $data/train/wav.scp -d $data/train/segments -d $data/train/utt2spk \
-O $exp/train_mfcc -O $data/train/feats.scp \
-n mfcc_train \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/train $exp/log/mfcc $exp/train_mfcc"

Stage is cached, skipping                                                       


In [93]:
dvc run \
-d $data/dev/wav.scp -d $data/dev/segments -d $data/dev/utt2spk \
-O $exp/dev_mfcc -O $data/dev/feats.scp \
-n mfcc_dev \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/dev $exp/log/mfcc $exp/dev_mfcc"

Running stage 'mfcc_dev' with command:                                          
	python3.6 /data/exp/kostya/ukroASR/features/make_mfcc.py /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/dev_mfcc
/bin/sh: 18: ./cmd.sh: [[: not found
steps/make_mfcc.sh /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/dev_mfcc
utils/validate_data_dir.sh: Successfully validated data-directory /data/exp/kostya/ukroASR/data/dev
steps/make_mfcc.sh [info]: segments file exists: using that.
Succeeded creating MFCC features for dev
Adding stage 'mfcc_dev' in 'dvc.yaml'                                           
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [94]:
git add dvc.yaml dvc.lock

In [23]:
# compute cmvn features

In [96]:
for split in train dev
do
dvc run \
-d $data/$split/feats.scp -d $data/$split/utt2spk \
-O $exp/${split}_cmvn -O $data/$split/cmvn.scp \
-n cmvn_${split} \
"python3.6 $UKROASR_ROOT/features/compute_cmvn_stats.py $data/$split $exp/log/cmvn $exp/${split}_cmvn"
done

Running stage 'cmvn_train' with command:                                        
	python3.6 /data/exp/kostya/ukroASR/features/compute_cmvn_stats.py /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/train_cmvn
/bin/sh: 18: ./cmd.sh: [[: not found
steps/compute_cmvn_stats.sh /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/train_cmvn
Succeeded creating CMVN stats for train
Adding stage 'cmvn_train' in 'dvc.yaml'                                         
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml
Running stage 'cmvn_dev' with command:                                          
	python3.6 /data/exp/kostya/ukroASR/features/compute_cmvn_stats.py /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/dev_cmvn
/bin/sh: 18: ./cmd.sh: [[: not found
steps/compute_cmvn_stats.sh /data/exp/kostya/ukr

In [97]:
git add dvc.yaml dvc.lock

# Sampling again

In [99]:
dvc run \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_2kshort/text -o $data/train_2kshort/utt2spk -o $data/train_2kshort/segments -o $data/train_2kshort/spk2utt -o $data/train_2kshort/wav.scp -o $data/train_2kshort/feats.scp -o $data/train_2kshort/cmvn.scp \
-n train_2kshort_data \
"$cmd_prefix utils/subset_data_dir.sh --shortest $data/train 2000 $data/train_2kshort"

Running stage 'train_2kshort_data' with command:                                
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest /data/exp/kostya/ukroASR/data/train 2000 /data/exp/kostya/ukroASR/data/train_2kshort
feat-to-len scp:/data/exp/kostya/ukroASR/data/train/feats.scp ark,t:/data/exp/kostya/ukroASR/data/train_2kshort/tmp.len 
utils/subset_data_dir.sh: reducing #utt from 12378 to 2000
Adding stage 'train_2kshort_data' in 'dvc.yaml'                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [101]:
dvc run \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_5k/text -o $data/train_5k/utt2spk -o $data/train_5k/segments -o $data/train_5k/spk2utt -o $data/train_5k/wav.scp -o $data/train_5k/feats.scp -o $data/train_5k/cmvn.scp \
-n train_5k_data \
"$cmd_prefix utils/subset_data_dir.sh $data/train 5000 $data/train_5k"

Running stage 'train_5k_data' with command:                                     
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh /data/exp/kostya/ukroASR/data/train 5000 /data/exp/kostya/ukroASR/data/train_5k
utils/subset_data_dir.sh: reducing #utt from 12378 to 5000
Adding stage 'train_5k_data' in 'dvc.yaml'                                      
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [104]:
dvc run \
-d $data/dev/text -d $data/dev/utt2spk -d $data/dev/segments -d $data/dev/spk2utt -d $data/dev/wav.scp -d $data/dev/feats.scp -d $data/dev/cmvn.scp \
-o $data/dev_0.75kshort/text -o $data/dev_0.75kshort/utt2spk -o $data/dev_0.75kshort/segments -o $data/dev_0.75kshort/spk2utt -o $data/dev_0.75kshort/wav.scp -o $data/dev_0.75kshort/feats.scp -o $data/dev_0.75kshort/cmvn.scp \
-n dev_075kshort_data \
"$cmd_prefix utils/subset_data_dir.sh --shortest $data/dev 750 $data/dev_0.75kshort"

Running stage 'dev_075kshort_data' with command:                                
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest /data/exp/kostya/ukroASR/data/dev 750 /data/exp/kostya/ukroASR/data/dev_0.75kshort
feat-to-len scp:/data/exp/kostya/ukroASR/data/dev/feats.scp ark,t:/data/exp/kostya/ukroASR/data/dev_0.75kshort/tmp.len 
utils/subset_data_dir.sh: reducing #utt from 879 to 750
Adding stage 'dev_075kshort_data' in 'dvc.yaml'                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [106]:
dvc run \
-d $data/dev_0.75kshort/text -d $data/dev_0.75kshort/utt2spk -d $data/dev_0.75kshort/segments -d $data/dev_0.75kshort/spk2utt -d $data/dev_0.75kshort/wav.scp -d $data/dev_0.75kshort/feats.scp -d $data/dev_0.75kshort/cmvn.scp \
-o $data/dev_0.15k/text -o $data/dev_0.15k/utt2spk -o $data/dev_0.15k/segments -o $data/dev_0.15k/spk2utt -o $data/dev_0.15k/wav.scp -o $data/dev_0.15k/feats.scp -o $data/dev_0.15k/cmvn.scp \
-n dev_015k_data \
"$cmd_prefix utils/subset_data_dir.sh $data/dev_0.75kshort 150 $data/dev_0.15k"

Running stage 'dev_015k_data' with command:                                     
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh /data/exp/kostya/ukroASR/data/dev_0.75kshort 150 /data/exp/kostya/ukroASR/data/dev_0.15k
utils/subset_data_dir.sh: reducing #utt from 750 to 150
Adding stage 'dev_015k_data' in 'dvc.yaml'                                      
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [107]:
git add dvc.yaml dvc.lock

# Training and alignment

# monophone

In [108]:
mkdir $exp/mono

mkdir: cannot create directory '/data/exp/kostya/ukroASR/exp/mono': File exists


: 1

In [110]:
dvc run \
-d $data/train_2kshort/feats.scp -d $data/train_2kshort/utt2spk -d $data/train_2kshort/text -d $data/train_2kshort/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones \
-o $exp/mono/final.mdl -o $exp/mono/tree -o $exp/mono/phones.txt \
-n mono \
"$cmd_prefix steps/train_mono.sh --boost-silence 1.25 --nj 8 $data/train_2kshort $data/lang $exp/mono"

Running stage 'mono' with command:                                              
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_mono.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_2kshort /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono
steps/train_mono.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_2kshort /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mon

In [39]:
git add mono.dvc exp/mono/.gitignore

In [112]:
dvc run \
-d $exp/mono/final.mdl -d $exp/mono/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/mono/graph/HCLG.fst -O $exp/mono/graph/words.txt \
-n mono_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang_ukr $exp/mono $exp/mono/graph"

Running stage 'mono_graph' with command:                                        
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono/graph
tree-info /data/exp/kostya/ukroASR/exp/mono/tree 
tree-info /data/exp/kostya/ukroASR/exp/mono/tree 
fstminimizeencoded 
fsttablecompose /data/exp/kostya/ukroASR/data/lang_ukr/L_disambig.fst /data/exp/kostya/ukroASR/data/lang_ukr/G.fst 
fstpushspecial 
fstdeterminizestar --use-log=true 
fstisstochastic /data/exp/kostya/ukroASR/data/lang_ukr/tmp/LG.fst 
-0.0360873 -0.03691
[info]: LG not stochastic.
fstcomposecontext --context-size=1 --central-position=0 --read-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/phones/disambig.int --write-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/tmp/disambig_ilabels_1_0.int /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_1_0.32655 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/LG.fs

In [115]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/mono/final.mdl -d $exp/mono/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/mono.json \
-n mono_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/mono/graph $data/dev_0.15k $exp/mono/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/mono/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/mono.json"

Running stage 'mono_metrics' with command:                                      
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/mono.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k
decode.sh: feature type is delta
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 53.3333333333% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/kostya/ukroASR/exp/

In [116]:
cat exp/metrics/mono.json

{'WER': 71.19}


In [8]:
cat exp/metrics/mono.json

{'WER': 72.83}


In [17]:
#base %WER 70.18
#2kshort %WER 75.96
#2kshort  + correct spk _2 %WER 75.41
#2kshort  + correct spk _2 no _I_B_S_E %WER 76.24
#2kshort  + correct spk _2 no _I_B_S_E small dev %WER 74.72 
#2kshort  + correct spk _2 no _I_B_S_E shared silence phones small dev %WER 75.25
#2kshort  + correct spk _2 no _I_B_S_E extra_questions stress %WER 75.70
######################################################################################################################
#2kshort  + 8 spk + no IBSE %WER 73.65
#2kshort  + 8 spk + no IBSE %WER extra_questions stress 74.34

# triphone

In [ ]:
# aligning train data

In [118]:
dvc run \
-d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/text -d $data/train_5k/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/mono/final.mdl -d $exp/mono/tree \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -O $f; done) \
-n mono_ali \
"$cmd_prefix steps/align_si.sh --boost-silence 1.25 --nj 8 $data/train_5k $data/lang $exp/mono $exp/mono_train_5k_ali"

Running stage 'mono_ali' with command:                                          
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/align_si.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/align_si.sh: feature type is delta
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train_5k using model from /data/exp/kostya/ukroASR/exp/mono, putting alignments in /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 50.461106656% of th

In [122]:
dvc run \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -d $f; done) -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones -d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/cmvn.scp \
-o $exp/tri1/final.mdl -o $exp/tri1/tree -o $exp/tri1/phones.txt \
-n tri1 \
"$cmd_prefix steps/train_deltas.sh --boost-silence 1.25 2000 10000 $data/train_5k $data/lang $exp/mono_train_5k_ali $exp/tri1; \
cp $exp/tri1/final.mdl $exp/tri1/final.mdl_copy; mv $exp/tri1/final.mdl_copy $exp/tri1/final.mdl"

Running stage 'tri1' with command:                                              
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_deltas.sh --boost-silence 1.25 2000 10000 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali /data/exp/kostya/ukroASR/exp/tri1; cp /data/exp/kostya/ukroASR/exp/tri1/final.mdl /data/exp/kostya/ukroASR/exp/tri1/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri1/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri1/final.mdl
steps/train_deltas.sh --boost-silence 1.25 2000 10000 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali /data/exp/kostya/ukroASR/exp/tri1
steps/train_deltas.sh: accumulating tree stats
steps/train_deltas.sh: getting questions for tree-building, via clustering
steps/train_deltas.sh: building the tree
WARNING (gmm-init-model[5.5.303~1-5a34a]:InitAmGmm():gmm-init-model.cc:55) Tree has pdf-id 

In [124]:
dvc run \
-d $exp/tri1/final.mdl -d $exp/tri1/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri1/graph/HCLG.fst -O $exp/tri1/graph/words.txt \
-n tri1_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang_ukr $exp/tri1 $exp/tri1/graph"

Running stage 'tri1_graph' with command:                                        
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1/graph
tree-info /data/exp/kostya/ukroASR/exp/tri1/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri1/tree 
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/phones/disambig.int --write-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/tmp/disambig_ilabels_3_1.int /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1.9608 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/LG.fst 
fstisstochastic /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_3_1.fst 
0 -0.03691
[info]: CLG not stochastic.
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/tri1/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1 /data/exp/kosty

In [126]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri1/final.mdl -d $exp/tri1/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/tri1.json \
-n tri1_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/tri1/graph $data/dev_0.15k $exp/tri1/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri1/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri1.json"

Running stage 'tri1_metrics' with command:                                      
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri1.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k
decode.sh: feature type is delta
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 49.3333333333% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/kostya/ukroASR/exp/

In [127]:
cat exp/metrics/tri1.json

{'WER': 54.45}


In [51]:
cat exp/metrics/tri1.json

{'WER': 56.63}


In [ ]:
# base %WER 60.95
# half + 2kshort %WER 61.70
# whole_train + 2kshort %WER 61.73
# half + 2kshort + correct spk_2 no _I_B_S_E short dev %WER 61.26
# half + 2kshort + correct spk_2 no _I_B_S_E shared silence phones %WER 63.56
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 61.51
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.72
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 56.28

# lda mllt

In [129]:
dvc run \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/tri1/final.mdl -d $exp/tri1/tree \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -O $f; done) \
-n tri_ali \
"$cmd_prefix steps/align_si.sh --nj 8 $data/train $data/lang $exp/tri1 $exp/tri1_train_ali"

Running stage 'tri_ali' with command:                                           
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/align_si.sh: feature type is delta
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train using model from /data/exp/kostya/ukroASR/exp/tri1, putting alignments in /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 50.2029550252% of the time at utterance end.  This may not be optimal.
steps/diagn

In [130]:
mkdir /data/exp/kostya/ukroASR/exp/tri2b

mkdir: cannot create directory '/data/exp/kostya/ukroASR/exp/tri2b': File exists


: 1

In [132]:
dvc run \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -d $f; done) -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri2b/final.mdl -o $exp/tri2b/final.mat -o $exp/tri2b/tree -o $exp/tri2b/phones.txt \
-n tri2b \
"$cmd_prefix steps/train_lda_mllt.sh --splice-opts \"--left-context=3 --right-context=3\" 2500 15000 $data/train $data/lang $exp/tri1_train_ali $exp/tri2b; \
cp $exp/tri2b/final.mdl $exp/tri2b/final.mdl_copy; mv $exp/tri2b/final.mdl_copy $exp/tri2b/final.mdl; \
cp $exp/tri2b/final.mat $exp/tri2b/final.mat_copy; mv $exp/tri2b/final.mat_copy $exp/tri2b/final.mat"

Running stage 'tri2b' with command:                                             
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_lda_mllt.sh --splice-opts "--left-context=3 --right-context=3" 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali /data/exp/kostya/ukroASR/exp/tri2b; cp /data/exp/kostya/ukroASR/exp/tri2b/final.mdl /data/exp/kostya/ukroASR/exp/tri2b/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri2b/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri2b/final.mdl; cp /data/exp/kostya/ukroASR/exp/tri2b/final.mat /data/exp/kostya/ukroASR/exp/tri2b/final.mat_copy; mv /data/exp/kostya/ukroASR/exp/tri2b/final.mat_copy /data/exp/kostya/ukroASR/exp/tri2b/final.mat
steps/train_lda_mllt.sh --splice-opts --left-context=3 --right-context=3 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali /data/exp/kostya/ukroASR/exp/tri2b


In [134]:
dvc run \
-d $exp/tri2b/final.mdl -d $exp/tri2b/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri2b/graph/HCLG.fst -O $exp/tri2b/graph/words.txt \
-n tri2b_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang_ukr $exp/tri2b $exp/tri2b/graph"

Running stage 'tri2b_graph' with command:                                       
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b/graph
tree-info /data/exp/kostya/ukroASR/exp/tri2b/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri2b/tree 
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/tri2b/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1 /data/exp/kostya/ukroASR/exp/tri2b/tree /data/exp/kostya/ukroASR/exp/tri2b/final.mdl 
fstrmepslocal 
fstrmsymbols /data/exp/kostya/ukroASR/exp/tri2b/graph/disambig_tid.int 
fstdeterminizestar --use-log=true 
fsttablecompose /data/exp/kostya/ukroASR/exp/tri2b/graph/Ha.fst /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_3_1.fst 
fstminimizeencoded 
fstisstochastic /data/exp/kostya/ukroASR/exp/tri2b/graph/HCLGa.fst 
0.578142 -0.104913
HCLGa is not stochastic

In [136]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/tri2b.json \
-n tri2b_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/tri2b/graph $data/dev_0.15k $exp/tri2b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri2b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri2b.json"

Running stage 'tri2b_metrics' with command:                                     
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri2b.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k
decode.sh: feature type is lda
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 47.3333333333% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/kostya/ukroAS

In [137]:
cat $exp/metrics/tri2b.json

{'WER': 53.15}


In [36]:
cat $exp/metrics/tri2b.json

{'WER': 55.49}


In [ ]:
#base %WER 68.48
# half + 2kshort %WER 68.22
# full + 2kshort + correct spk %WER 67.81
# full + 2kshort + correct spk _2 %WER 65.25
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 64.77
# 2kshort + half + correct spk_2 + no _I_B_S_E shared silence phones  %WER 67.73
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 64.91
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.31
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 57.05

# LDA + MLLT + SAT

In [139]:
dvc run \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/full.mat -d $exp/tri2b/tree \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -O $f; done) \
-n tri2b_ali \
"$cmd_prefix steps/align_si.sh --nj 8 $data/train $data/lang $exp/tri2b $exp/tri2b_train_ali"

Running stage 'tri2b_ali' with command:                                         
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/align_si.sh: feature type is lda
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train using model from /data/exp/kostya/ukroASR/exp/tri2b, putting alignments in /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 48.3503981797% of the time at utterance end.  This may not be optimal.
steps/

In [141]:
dvc run \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -d $f; done) -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri3b/final.mdl -o $exp/tri3b/final.mat -o $exp/tri3b/tree -o $exp/tri3b/phones.txt \
-n tri3b \
"$cmd_prefix steps/train_sat.sh 2500 15000 $data/train $data/lang $exp/tri2b_train_ali $exp/tri3b; \
cp $exp/tri3b/final.mdl $exp/tri3b/final.mdl_copy; mv $exp/tri3b/final.mdl_copy $exp/tri3b/final.mdl; \
cp $exp/tri3b/final.mat $exp/tri3b/final.mat_copy; mv $exp/tri3b/final.mat_copy $exp/tri3b/final.mat"

Running stage 'tri3b' with command:                                             
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_sat.sh 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali /data/exp/kostya/ukroASR/exp/tri3b; cp /data/exp/kostya/ukroASR/exp/tri3b/final.mdl /data/exp/kostya/ukroASR/exp/tri3b/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri3b/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri3b/final.mdl; cp /data/exp/kostya/ukroASR/exp/tri3b/final.mat /data/exp/kostya/ukroASR/exp/tri3b/final.mat_copy; mv /data/exp/kostya/ukroASR/exp/tri3b/final.mat_copy /data/exp/kostya/ukroASR/exp/tri3b/final.mat
steps/train_sat.sh 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali /data/exp/kostya/ukroASR/exp/tri3b
steps/train_sat.sh: feature type is lda
steps/train_sat.sh: obtaining initial fMLLR transforms since not pre

In [143]:
dvc run \
-d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri3b/graph/HCLG.fst -O $exp/tri3b/graph/words.txt \
-n tri3b_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang $exp/tri3b $exp/tri3b/graph"

Running stage 'tri3b_graph' with command:                                       
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri3b /data/exp/kostya/ukroASR/exp/tri3b/graph
tree-info /data/exp/kostya/ukroASR/exp/tri3b/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri3b/tree 
fstpushspecial 
fstdeterminizestar --use-log=true 
fsttablecompose /data/exp/kostya/ukroASR/data/lang/L_disambig.fst /data/exp/kostya/ukroASR/data/lang/G.fst 
fstminimizeencoded 
fstisstochastic /data/exp/kostya/ukroASR/data/lang/tmp/LG.fst 
-0.046394 -0.047274
[info]: LG not stochastic.
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=/data/exp/kostya/ukroASR/data/lang/phones/disambig.int --write-disambig-syms=/data/exp/kostya/ukroASR/data/lang/tmp/disambig_ilabels_3_1.int /data/exp/kostya/ukroASR/data/lang/tmp/ilabels_3_1.30811 /data/exp/kostya/ukroASR/data/lang/tmp/LG.fst 
fstisstochastic /data/exp

In [146]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $exp/tri3b/final.mat -d $exp/tri3b/graph/HCLG.fst -d $data/lang/words.txt -d $data/lang/phones \
-m $exp/metrics/tri3b.json \
-n tri3b_metrics \
"$cmd_prefix steps/decode_fmllr.sh --nj 4 $exp/tri3b/graph $data/dev_0.15k $exp/tri3b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri3b.json"

Running stage 'tri3b_metrics' with command:                                     
	cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode_fmllr.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri3b.json
steps/decode_fmllr.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k
steps/decode.sh --scoring-opts  --num-threads 1 --skip-scoring false --acwt 0.083333 --nj 4 --cmd run.pl --beam 10.0 --model /data/exp/kostya/ukroASR/exp/tri3b/final.alimdl --max-active 2000 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k.si
decode.sh: feature type is lda
steps/diagnosti

In [148]:
cat $exp/metrics/tri3b.json

{'WER': 31.89}


In [54]:
cat $exp/metrics/tri3b.json

{'WER': 45.75}


In [ ]:
# base %WER 54.14
# half + 2kshort %WER 53.50
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 50.55
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 50.85
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 47.46
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 46.80